In [2]:
# =====================================
# 🎯 Google Colab：多物件偵測（外框 + 純文字）
# 同類別：框顏色 = 文字顏色
# 無任何文字背景
# =====================================

# 0️⃣ 安裝套件（一定要在 import 前）
!pip install -q ultralytics opencv-python

# 1️⃣ 匯入套件
import os
import cv2
from ultralytics import YOLO
from google.colab import drive
from IPython.display import HTML, display

# 2️⃣ 掛載 Google Drive（如不用可保留，無影響）
drive.mount('/content/drive', force_remount=True)

# 3️⃣ ======【更換影片只改這裡】======
# 範例 1：Colab 測試影片
VIDEO_SOURCE = "/content/drive/MyDrive/20251226AI/images.jpg"

# 如果你要用 Drive 上的影片，改成例如：
# VIDEO_SOURCE = "/content/drive/MyDrive/videos/my_video.mp4"

# 如果是第一次用測試影片，下載它
if not os.path.exists(VIDEO_SOURCE):
    !wget -q -O /content/sample_video.avi https://github.com/opencv/opencv/raw/master/samples/data/vtest.avi

# 4️⃣ 載入模型（YOLOv8x，80 類）
model = YOLO("yolov8x.pt")

# 5️⃣ 開啟影片
cap = cv2.VideoCapture(VIDEO_SOURCE)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_path = "/content/drive/MyDrive/yolo_result_no_bg_text.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# 6️⃣ 高對比顏色（固定，避免太暗）
COLOR_PALETTE = [
    (255, 255, 255),  # 白
    (0, 255, 255),    # 黃
    (255, 200, 0),    # 淺藍
    (0, 255, 0),      # 綠
    (255, 0, 255),    # 紫
    (255, 128, 0),    # 橘
]

def get_color(cls_id):
    return COLOR_PALETTE[cls_id % len(COLOR_PALETTE)]

# 7️⃣ 偵測並畫框（純文字、無背景）
for result in model(source=VIDEO_SOURCE, stream=True):
    frame = result.orig_img.copy()

    if result.boxes is None:
        out.write(frame)
        continue

    for box in result.boxes:
        cls_id = int(box.cls)
        label = model.names[cls_id]
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        color = get_color(cls_id)

        # 外框
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # 純文字（無背景）
        cv2.putText(
            frame,
            label,  # 若要信心度：f"{label} {conf:.2f}"
            (x1, max(y1 - 10, 20)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2,
            cv2.LINE_AA
        )

    out.write(frame)

cap.release()
out.release()

# 8️⃣ 顯示結果影片
display(HTML(f"""
<video width="720" controls>
  <source src="{output_path}" type="video/mp4">
</video>
"""))

print("✅ 完成，輸出影片位置：", output_path)


Mounted at /content/drive

image 1/1 /content/drive/MyDrive/20251226AI/images.jpg: 480x640 2 persons, 4 cars, 1 handbag, 114.9ms
Speed: 12.1ms preprocess, 114.9ms inference, 48.2ms postprocess per image at shape (1, 3, 480, 640)


✅ 完成，輸出影片位置： /content/drive/MyDrive/yolo_result_no_bg_text.mp4
